## Pages List (Buy)

In [185]:
import requests
from bs4 import BeautifulSoup

pages_list = []
for i in range(1, 839):
    pages_list.append('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')
    #print('Paginator #' + str(i) + ": " + pages_list[i-1])

## Properties Pages List (Buy)

In [186]:
paginator_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')
properties_number = soup.find(id='lblNumInm').get_text()
#print (properties_number + 'properties in total\n')
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    #print('Property #' + str(i) + ": " + str(url_property))
    properties_pages_list.append(url_property)

## Property Scraping (Old Properties)

In [164]:
paginator_url = 'https://www.fincaraiz.com.co/apartamento-en-venta/medellin/calasanz-det-4966404.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Detail Header
detail_header = soup.find('div', class_='row detailheader')
property_price = detail_header.find('div', class_='price').find('h2').get_text()

# Extract Image Content
image_content = soup.find('div', class_='row imageContent')
property_use = image_content.find('div', id='ctl00_phMasterPage_cAdvert_ucMultimedias_icoUsed').get_text()

# Extract Small Features
small_features = soup.find('div', class_='features clearfix')
property_sqrt_meters = small_features.find('span', class_='advertSurface').get_text().split()
property_sqrt_meters = float(" ".join(property_sqrt_meters)[0:-3].replace(",", "."))
property_rooms = small_features.find('span', class_='advertRooms').get_text().split()
property_rooms = int(" ".join(property_rooms)[-1])
property_baths = small_features.find('span', class_='advertBaths').get_text().split()
property_baths = int(" ".join(property_baths)[-1])
property_park = small_features.find('span', class_='advertGarages').get_text().split()
property_park = " ".join(property_park)[-1]

# Estract Big Features
big_features = soup.find('ul', class_='boxcube')
property_private_area = ''
property_const_area = ''
property_sqrt_meters_price = ''
property_stratum = ''
property_feature_status = ''
property_antiquity = ''
property_floor = ''
property_sector = ''
property_weather = ''
property_admon = ''
for li in big_features.find_all('li'):
    value = li.get_text().split()
    value = " ".join(value)
    if(value.find('Área privada') == 0):
        property_private_area = value.replace('Área privada: ', '')[0:-3]
    elif(value.find('Área Const') == 0):
        property_const_area = value.replace('Área Const.: ', '')[0:-3]
    elif(value.find('Precio m²') == 0):
        property_sqrt_meters_price = value.replace('Precio m²: ', '')[0:-3]
    elif(value.find('Estrato') == 0):
        property_stratum = value.replace('Estrato: ', '')[0]
    elif(value.find('Estado') == 0):
        property_feature_status = value.replace('Estado: ', '')
    elif(value.find('Antigüedad') == 0):
        property_antiquity = value.replace('Antigüedad: ', '')
    elif(value.find('Piso No') == 0):
        property_floor = value.replace('Piso No: ', '')[0:-1]
    elif(value.find('Sector') == 0):
        property_sector = value.replace('Sector: ', '')
    elif(value.find('Clima') == 0):
        property_weather = value.replace('Clima: ', '')
    elif(value.find('Admón:') == 0):
        property_admon = value.replace('Admón: ', '')

property_features = []
features_object = {
    'squareMeters': property_sqrt_meters,
    'rooms': property_rooms,
    'bathrooms': property_baths,
    'parkingLot': property_park,
    'privateArea': property_private_area,
    'constructionArea': property_const_area,
    'squareMetersPrice': property_sqrt_meters_price,
    'stratum': property_stratum,
    'status': property_feature_status,
    'antiquity': property_antiquity,
    'floors': property_floor,
    'weather': property_weather,
    'admonPrice': property_admon
}
property_features.append(features_object)

# Extract Description
description = soup.find('div', class_='description')
property_code = int(description.find('span').find('b').get_text())
property_description = description.find('p').text

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for li in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

## Property Scraping (New Properties)

In [189]:
paginator_url = 'https://www.fincaraiz.com.co/palermo/medellin/proyecto-nuevo-det-2393445.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Header
detail_header = soup.find('div', class_='row detailheader')

detail_title = detail_header.find('div', class_='title')
property_name_project = detail_title.find('div', class_='box').find('h1').text
property_address_project = detail_title.find('span', class_='address').text

detail_price = detail_header.find('div', class_='price')
property_min_price = ''
property_max_price = ''
array_prices = []
for price in detail_price.find_all('label'):
    array_prices.append(price.text)
property_min_price = array_prices[0].split()
property_min_price = " ".join(property_min_price)
property_max_price = array_prices[1].split()
property_max_price = " ".join(property_max_price)

property_price = []
price_object = {
    'minPrice': property_min_price,
    'maxPrice': property_max_price
}
property_price.append(price_object)

# Extract Small Features
small_features = soup.find('div', class_='features clearfix')
property_sqrt_meters = small_features.find('span', class_='advertSurface').get_text().split()
property_sqrt_meters = " ".join(property_sqrt_meters)[0:-3].replace(",", ".")
property_rooms = small_features.find('span', class_='advertRooms').get_text().split()
property_rooms = int(" ".join(property_rooms)[-1])
property_baths = small_features.find('span', class_='advertBaths').get_text().split()
property_baths = int(" ".join(property_baths)[-1])
property_park = small_features.find('span', class_='advertGarages').get_text().split()
property_park = " ".join(property_park)[-1]

# Estract Big Features
big_features = soup.find('ul', class_='boxcube')
property_private_area = ''
property_const_area = ''
property_sqrt_meters_price = ''
property_stratum = ''
property_feature_status = ''
property_antiquity = ''
property_floor = ''
property_sector = ''
property_weather = ''
property_admon = ''
for li in big_features.find_all('li'):
    value = li.get_text().split()
    value = " ".join(value)
    if(value.find('Área privada') == 0):
        property_private_area = value.replace('Área privada: ', '')[0:-3]
    elif(value.find('Área Const') == 0):
        property_const_area = value.replace('Área Const.: ', '')[0:-3]
    elif(value.find('Precio m²') == 0):
        property_sqrt_meters_price = value.replace('Precio m²: ', '')[0:-3]
    elif(value.find('Estrato') == 0):
        property_stratum = value.replace('Estrato: ', '')[0]
    elif(value.find('Estado') == 0):
        property_feature_status = value.replace('Estado: ', '')
    elif(value.find('Antigüedad') == 0):
        property_antiquity = value.replace('Antigüedad: ', '')
    elif(value.find('Piso No') == 0):
        property_floor = value.replace('Piso No: ', '')[0:-1]
    elif(value.find('Sector') == 0):
        property_sector = value.replace('Sector: ', '')
    elif(value.find('Clima') == 0):
        property_weather = value.replace('Clima: ', '')
    elif(value.find('Admón:') == 0):
        property_admon = value.replace('Admón: ', '')
        
# Extract Description
description = soup.find('div', class_='description')
property_code = int(description.find('span').find('b').get_text())
property_description = description.find('p').text

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for li in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

# Extract Offers Types
offers_type = soup.find('div', id='typology')
table = offers_type.find('table')
#print(table.prettify())

## Create JSON

In [191]:
import json

#buy_old_property_dict = {
    #'code': property_code,
    #'status': 'Venta',
    #'use': property_use,
    #'price': property_price,
    #'country': 'Colombia',
    #'department': 'Antioquia',
    #'city': 'Medellin',
    #'sector': property_sector,
    #'description': property_description,
    #'features': property_features,
    #'interiorFeatures': property_interior_features,
    #'exteriorFeatures': property_exterior_features,
    #'sectorFeatures': property_sector_features
#

buy_new_property_dict = {
    'code': property_code,
    'nameProject': property_name_project,
    'status': 'Venta',
    'use': 'Nuevo',
    'price': property_price,
    'country': 'Colombia',
    'department': 'Antioquia',
    'city': 'Medellin',
    'sector': property_sector,
    'description': property_description,
    'features': property_features,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features
}

print(json.dumps(buy_new_property_dict, indent=4))

{
    "code": 2393445,
    "nameProject": "Palermo",
    "status": "Venta",
    "use": "Nuevo",
    "price": [
        {
            "minPrice": "$ 292.633.175",
            "maxPrice": "$ 504.651.335"
        }
    ],
    "country": "Colombia",
    "department": "Antioquia",
    "city": "Medellin",
    "sector": "Loma del Indio",
    "description": "PALERMO, donde las cosas importantes se encuentran muy cerca. Proyecto de apartamentos muy exclusivos, contiguo a la Iglesia SAN JOSE DE NAZARETH.  Ac\u00e1 encontrar\u00e1 todo a la mano porque en nuestro d\u00eda a d\u00eda necesitamos de bancos, restaurantes, supermercados, estaciones de gasolina que nos hacen mas f\u00e1cil la vida. En el \u00faltimo piso de la torre tendremos un agradable Sky Club con jacuzzi para 10 personas, gimnasio, sal\u00f3n social y una espectacular vista..",
    "features": [
        {
            "squareMeters": 84.0,
            "rooms": 3,
            "bathrooms": 2,
            "parkingLot": "1",
         